![](img/40_analysis.png)

##### Проверка <u>элементарных функций</u>

*Математические преобразования*

In [2]:
from __init__ import *
a = kf.get_vars("a", 3, numb=False)
kf.my_print(f"Screw-symmetrix matrix [a]x:", bold=True)
display(kf.get_antisymmetric_matrix(a))

Screw-symmetrix matrix [a]x:


Matrix([
[   0, -a_z,  a_y],
[ a_z,    0, -a_x],
[-a_y,  a_x,    0]])

*Кватернионы*

In [15]:
from __init__ import *
q1, q2 = kf.get_vars("q^1", 4, numb=False), kf.get_vars("q^2", 4, numb=False)
q1v, q2v = Matrix(q1[1:4]), Matrix(q2[1:4])

kf.my_print(f"Скалярная часть (вручную):", bold=True)
q12_scalar = q1[0]*q2[0] - q1v.dot(q2v)
display(q12_scalar)
kf.my_print(f"Векторная часть (вручную):", bold=True)
q12_vec = q1v*q2[0] + q2v*q1[0] + q1v.cross(q2v)
display(q12_vec)
kf.my_print(f"Умножение кватернионов:", bold=True)
q12 = kf.q_dot(q1, q2)
display(q12)
kf.my_print(f"Одно равно другому: {q12_scalar == q12[0]}, {q12_vec == Matrix(q12[1:4])}", bold=True)

Скалярная часть (вручную):


q^1_0*q^2_0 - q^1_x*q^2_x - q^1_y*q^2_y - q^1_z*q^2_z

Векторная часть (вручную):


Matrix([
[q^1_0*q^2_x + q^1_x*q^2_0 + q^1_y*q^2_z - q^1_z*q^2_y],
[q^1_0*q^2_y - q^1_x*q^2_z + q^1_y*q^2_0 + q^1_z*q^2_x],
[q^1_0*q^2_z + q^1_x*q^2_y - q^1_y*q^2_x + q^1_z*q^2_0]])

Умножение кватернионов:


Matrix([
[q^1_0*q^2_0 - q^1_x*q^2_x - q^1_y*q^2_y - q^1_z*q^2_z],
[q^1_0*q^2_x + q^1_x*q^2_0 + q^1_y*q^2_z - q^1_z*q^2_y],
[q^1_0*q^2_y - q^1_x*q^2_z + q^1_y*q^2_0 + q^1_z*q^2_x],
[q^1_0*q^2_z + q^1_x*q^2_y - q^1_y*q^2_x + q^1_z*q^2_0]])

Одно равно другому: True, True


Поворот на $90^\circ$ вокруг оси $Oz$: 
$$\lambda = [\cos\frac{\varphi}{2}, 0, 0, \sin\frac{\varphi}{2}]$$
$$x\to y, \hskip20px y \to -x$$

In [10]:
q = np.quaternion(1, 0, 0, 1).normalized()
kf.my_print(f"Кватернион:", bold=True)
print(q)

A = kf.quart2dcm(q1)
A = np.round(A).astype(np.int64)
kf.my_print(f"Матрица поворота (соответствующая):", bold=True)
print(A)

kf.my_print(f"Повороты векторов:", bold=True)
for a in [np.array([1, 0, 0]), 
          np.array([0, 1, 0])]:
    print(f"{a} -> {A @ a}")

Кватернион:
quaternion(0.707106781186547, 0, 0, 0.707106781186547)
Матрица поворота (соответствующая):
[[ 0 -1  0]
 [ 1  0  0]
 [ 0  0  1]]
Повороты векторов:
[1 0 0] -> [0 1 0]
[0 1 0] -> [-1  0  0]


*Угол поворота матрицы* $$\cos\alpha = \frac{1}{2} [ tr(S) - 1]$$ 

In [4]:
from __init__ import *
q = np.quaternion(*np.random.uniform(-1, 1, 4)).normalized()
S = quaternion.as_rotation_matrix(q)

kf.my_print(f"Сравнение формулы и расчёта по модулю:", bold=True)
cos_alpha = kf.clip((np.trace(S) - 1) / 2, -1, 1)
print(f"Косинус по формуле: {cos_alpha}")
print(f"Косинус из модуля: {np.cos(quaternion.np.angle_of_rotor(quaternion.from_rotation_matrix(S)))}")

phi = var('varphi')
r = kf.get_vars(name='r', n=3, numb=False)

for q in [kf.vec2quat(kf.get_vars(name='q', n=3, numb=False)),
          kf.get_q_Rodrigue_Hamilton(phi=phi, r=r, symbol=True)]:
    kf.my_print(f"При кватернионе", bold=True)
    display(q.T)
    M = kf.quart2dcm(q)
    cos_s = kf.matrix2angle(M)
    print(f"косинус:")
    display(cos_s.simplify())

Сравнение формулы и расчёта по модулю:
Косинус по формуле: -0.32708460215879775
Косинус из модуля: -0.32708460215879787
При кватернионе


Matrix([[sqrt(-q_x**2 - q_y**2 - q_z**2 + 1), q_x, q_y, q_z]])

косинус:


-2*q_x**2 - 2*q_y**2 - 2*q_z**2 + 1

При кватернионе


Matrix([[cos(varphi/2), r_x*sin(varphi/2)/sqrt(r_x**2 + r_y**2 + r_z**2), r_y*sin(varphi/2)/sqrt(r_x**2 + r_y**2 + r_z**2), r_z*sin(varphi/2)/sqrt(r_x**2 + r_y**2 + r_z**2)]])

косинус:


cos(varphi)

##### Проверка <u>фильтра Калмана</u>

In [3]:
from __init__ import *
o = kf.init(symbolic=True)
t, ω, μ, ρ = o.p.t, o.W_ORB, o.MU, o.RHO 

o.f.r_orf, o.f.v_orf, o.f.q, o.f.w_brf = get_state_vector(func=kf.get_func, obj='d', n=1)
o.c.r_orf, o.c.v_orf, o.c.q, o.c.w_brf = get_state_vector(func=kf.get_func, obj='c', n=1)

for i in range(1):  # ПОТОМ ПОСТАВИТЬ 2, ПРОВЕРИТЬ ОЦЕНКУ УГЛОВОГО ДВИЖЕНИЯ
    kf.my_print(f"Оценка орбитального{['', ' и углового'][i]} движения", color='m', bold=True)

    kf.my_print(f"Матрица линеаризованной динамики Ф:", bold=True)
    display(o.p.k.get_Phi(w=o.f.w_brf, w0=o.W_ORB))

    # Поправки на угловое движение
    o.NAVIGATION_ANGLES = True

Оценка орбитального движения
Матрица линеаризованной динамики Ф:


Matrix([
[1.0,          0,           0,       dt,   0,         0],
[  0,        1.0,           0,        0,  dt,         0],
[  0,          0,         1.0,        0,   0,        dt],
[  0,          0,           0,      1.0,   0, -2*dt*w_0],
[  0, -dt*w_0**2,           0,        0, 1.0,         0],
[  0,          0, 3*dt*w_0**2, 2*dt*w_0,   0,       1.0]])